# Model training lab

This is the notebook for loading and training models.
Furthermore it provides simple documentation for different approaches used for training a model.

Run the command below to see command-completion on pressing `TAB`.

## Prerequisits

In [6]:
# Imports
import os
import warnings
import tools
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.layers import Bidirectional

# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Root CSV files directory
dirname = "./data/"

# Constant frame count.
frames = 100

## Preparation Stage
### Load data and normalize
For training it's required to extend/reduce every dataset to n frames, where n is `frames`.

In [7]:
listfile = os.listdir(dirname)
data = []
for wordname in listfile:
    if wordname == ".DS_Store":
        continue
    for csv in os.listdir(dirname + wordname):
        filepath = os.path.join(dirname, wordname, csv)
        content = pd.read_csv(filepath, sep=';')
        content = content.reindex(list(range(0, frames)), fill_value=0.0)
        content.fillna(0.0, inplace = True) 
        data.append((wordname, content))


In [8]:
# Print the content of the first dataframe
data[0][1]

,face_x,face_y,landmark_x_1,landmark_y_1,landmark_x_2,landmark_y_2,landmark_x_3,landmark_y_3,landmark_x_4,landmark_y_4,...,landmark_x_38,landmark_y_38,landmark_x_39,landmark_y_39,landmark_x_40,landmark_y_40,landmark_x_41,landmark_y_41,landmark_x_42,landmark_y_42
0,0.513086,0.317546,0.734630,0.730928,0.682608,0.740892,0.647388,0.753325,0.608517,0.790681,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,0.514772,0.319561,0.287116,0.713952,0.332434,0.729370,0.377520,0.710072,0.398559,0.688866,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
2,0.509703,0.322235,0.286905,0.676018,0.331073,0.678926,0.375002,0.663713,0.402209,0.645312,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,0.511009,0.322432,0.300466,0.639038,0.346397,0.632383,0.380598,0.614600,0.408101,0.596380,...,0.303501,0.466568,0.271055,0.517031,0.258359,0.506262,0.254735,0.508821,0.256895,0.50613
4,0.511274,0.322397,0.317917,0.615976,0.366193,0.604277,0.401965,0.582840,0.425050,0.563845,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
96,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
97,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
98,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


### Split data
Split the dataset up into the following segments:
1. Training Data: 60%
2. Validation Data: 20%
3. Test Data: 20%


In [9]:
features = [n[1] for n in data]
features = [f.to_numpy() for f in features]
labels = [n[0] for n in data]
x_train, x_val, y_train, y_val = train_test_split(features, labels, test_size=0.40, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.50, random_state=42)

In [10]:
# Display data distribution
print("Total:", len(labels))
print("Training:", len(y_train), len(y_train) / len(labels) * 100)
print("Val:", len(y_val), len(y_val) / len(labels) * 100)
print("Test:", len(y_test), len(y_test) / len(labels) * 100)

Total: 347
Training: 208 59.94236311239193
Val: 69 19.88472622478386
Test: 70 20.172910662824208


### Tokenize (One Hot)

In [11]:
import tools
tokenizer = tools.tokenize(dirname)
print(tokenizer.word_index)
encoded_train=tokenizer.texts_to_sequences([y_train])[0]
encoded_val=tokenizer.texts_to_sequences([y_val])[0]
encoded_test=tokenizer.texts_to_sequences([y_test])[0]
y_train = to_categorical(encoded_train)
y_val = to_categorical(encoded_val)
y_test = to_categorical(encoded_test)
print(y_train)

{'computer': 1, 'deutschland': 2, 'hallo': 3, 'welt': 4}
[[0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]]


In [12]:
# Making numpy arrays
x_train=np.array(x_train)
y_train=np.array(y_train)
x_val=np.array(x_val)
y_val=np.array(y_val)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [13]:
print(x_train)

[[[0.457003 0.339148 0.287502 ... 0.       0.       0.      ]
  [0.456536 0.347016 0.329147 ... 0.       0.       0.      ]
  [0.455053 0.343733 0.341495 ... 0.       0.       0.      ]
  ...
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]]

 [[0.452626 0.295807 0.28652  ... 0.       0.       0.      ]
  [0.457159 0.297622 0.343002 ... 0.       0.       0.      ]
  [0.459779 0.297895 0.340855 ... 0.       0.       0.      ]
  ...
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]]

 [[0.492527 0.316002 0.292566 ... 0.       0.       0.      ]
  [0.499061 0.31738  0.270048 ... 0.       0.       0.      ]
  [0.499876 0.322614 0.269635 ... 0.       0.       0.      ]
  ...
  [0.       0.       0.       ... 0.       0.   

## Training Stage
Configure the model and train it.

Metrics:
![accuracy](assets/accuracy.png)
![precision and recall formula](assets/precision_recall_formula.png)
![precision and recall](assets/precision_recall.png)


### Configure model
Configures the model with the specified parameters.

#### LSTM

In [14]:
model = Sequential()
model.add(layers.LSTM(64, return_sequences=True,
               input_shape=(100, 86)))  # returns a sequence of vectors of dimension 32
model.add(layers.LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(layers.LSTM(32))  # return a single vector of dimension 32
model.add(layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

### or
#### Bidirectional LSTM

In [ ]:
model = Sequential()
model.add(Bidirectional(layers.LSTM(64, return_sequences=True), input_shape=(100, 86)))
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

### or
#### Simple RNN

In [ ]:
model = Sequential()
model.add(SimpleRNN(units=64, input_shape=(200, 42)))
model.add(Dense(64, activation="softmax")) #softmax, linear 어떤걸 기준으로 하지
model.add(Dense(128, activation="linear")) #softmax, linear 어떤걸 기준으로 하지
model.add(Dense(21))
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

### Train

In [15]:
history=model.fit(x_train,y_train,epochs=100,batch_size=32,validation_data=(x_val,y_val))

Train on 208 samples, validate on 69 samples
Epoch 1/100
208/208 [==============================] - 7s 34ms/sample - loss: 1.5280 - accuracy: 0.2500 - precision: 0.8000 - recall: 0.0192 - val_loss: 1.4190 - val_accuracy: 0.2319 - val_precision: 0.7222 - val_recall: 0.1884
Epoch 2/100
208/208 [==============================] - 1s 3ms/sample - loss: 1.3257 - accuracy: 0.3942 - precision: 0.8261 - recall: 0.1827 - val_loss: 1.2836 - val_accuracy: 0.4348 - val_precision: 1.0000 - val_recall: 0.1304
Epoch 3/100
208/208 [==============================] - 1s 3ms/sample - loss: 1.2159 - accuracy: 0.4375 - precision: 0.8372 - recall: 0.1731 - val_loss: 1.1234 - val_accuracy: 0.4783 - val_precision: 0.7778 - val_recall: 0.2029
Epoch 4/100
208/208 [==============================] - 1s 3ms/sample - loss: 1.0738 - accuracy: 0.4952 - precision: 0.8269 - recall: 0.2067 - val_loss: 1.0613 - val_accuracy: 0.4348 - val_precision: 0.8235 - val_recall: 0.2029
Epoch 5/100
208/208 [=========================

Epoch 37/100
208/208 [==============================] - 1s 3ms/sample - loss: 0.7220 - accuracy: 0.6442 - precision: 0.7388 - recall: 0.4760 - val_loss: 0.6752 - val_accuracy: 0.6957 - val_precision: 0.7419 - val_recall: 0.6667
Epoch 38/100
208/208 [==============================] - 1s 3ms/sample - loss: 0.6807 - accuracy: 0.6490 - precision: 0.7422 - recall: 0.4567 - val_loss: 0.6748 - val_accuracy: 0.7246 - val_precision: 0.8919 - val_recall: 0.4783
Epoch 39/100
208/208 [==============================] - 1s 4ms/sample - loss: 0.6669 - accuracy: 0.6587 - precision: 0.7535 - recall: 0.5144 - val_loss: 0.6420 - val_accuracy: 0.6522 - val_precision: 0.9394 - val_recall: 0.4493
Epoch 40/100
208/208 [==============================] - 1s 3ms/sample - loss: 0.5888 - accuracy: 0.6923 - precision: 0.7920 - recall: 0.4760 - val_loss: 0.6288 - val_accuracy: 0.7391 - val_precision: 0.9412 - val_recall: 0.4638
Epoch 41/100
208/208 [==============================] - 1s 3ms/sample - loss: 0.8513 - a

Epoch 73/100
208/208 [==============================] - 1s 3ms/sample - loss: 0.4141 - accuracy: 0.8798 - precision: 0.8798 - recall: 0.8798 - val_loss: 0.4567 - val_accuracy: 0.8696 - val_precision: 0.8696 - val_recall: 0.8696
Epoch 74/100
208/208 [==============================] - 1s 3ms/sample - loss: 0.4558 - accuracy: 0.8798 - precision: 0.8883 - recall: 0.8798 - val_loss: 0.7630 - val_accuracy: 0.8261 - val_precision: 0.8261 - val_recall: 0.8261
Epoch 75/100
208/208 [==============================] - 1s 3ms/sample - loss: 0.5377 - accuracy: 0.8702 - precision: 0.8702 - recall: 0.8702 - val_loss: 0.4209 - val_accuracy: 0.9130 - val_precision: 0.9130 - val_recall: 0.9130
Epoch 76/100
208/208 [==============================] - 1s 3ms/sample - loss: 0.4047 - accuracy: 0.8798 - precision: 0.8798 - recall: 0.8798 - val_loss: 0.5155 - val_accuracy: 0.8986 - val_precision: 0.8986 - val_recall: 0.8986
Epoch 77/100
208/208 [==============================] - 1s 3ms/sample - loss: 0.3952 - a

### Evaluate

In [16]:
model.evaluate(x=x_test, y=y_test, batch_size=32)

70/70 [==============================] - 0s 1ms/sample - loss: 0.4857 - accuracy: 0.8571 - precision: 0.8571 - recall: 0.8571


[0.48569512718490193, 0.85714287, 0.85714287, 0.85714287]

### Save model

In [17]:
model.save("sign_lang_recognition.h5")